# Preview Analysis data

This script surfaces analysis data to ascertain its value

In [26]:
import bote 
import json
import pandas as pd

alt.renderers.enable('mimetype')

conn, cursor = bote.db_connect()

cursor.execute("SELECT * FROM ask where analysis is not null")
data = cursor.fetchall()


df = pd.DataFrame(data)
df.columns = [desc[0] for desc in cursor.description]

invalid_rows = []
for i, val in df['analysis'].items():
    try:
        json.loads(val)
    except (json.JSONDecodeError, TypeError):
        invalid_rows.append(i)

df = df.drop(invalid_rows)

df['analysis'] = df['analysis'].apply(json.loads)
df_analysis = pd.json_normalize(df['analysis'])
df = df.drop("embedding", axis=1)

def print_value_counts():
    for column in df_analysis.columns:
        print(f"\nColumn: {column}")
        print(df_analysis[column].value_counts())

def print_df():
    pd.set_option('display.max_colwidth', 1000)
    pd.set_option('display.max_rows', None)
    print(df)
    
html_output = df.to_html()

# Specify the file path where you want to save the HTML file
output_file_path = "output.html"

# Write the HTML representation to the file
with open(output_file_path, 'w') as f:
    f.write(html_output)

    
#df_analysis['advice_type'].unique()
#df_analysis['advice_type'].value_counts()

In [17]:
import bote 

conn, cursor = bote.db_connect()
#ask = bote.random_ask(cursor)

#bote.load_random_dicts()
#bote.embed_asks()
#bote.moderate_asks()
#bote.analyze_asks()

#bote.get_ask("MGlpMj2TunU")
